In [ ]:
import torch as th
import torch.nn as nn

from typing import Tuple

In [ ]:
new_block = nn.Sequential(
    nn.ConvTranspose2d(
        32, 32,
        kernel_size=(3, 3),
        stride=(2, 2),
        padding=(1, 1),
        output_padding=(1, 1)
    ),
    nn.ReLU(),

    nn.ConvTranspose2d(
        32, 24,
        kernel_size=(3, 3),
        stride=(1, 1),
        padding=(1, 1)
    ),
    nn.ReLU(),

    nn.ConvTranspose2d(
        24, 1,
        kernel_size=(1, 1),
        stride=(1, 1),
        padding=(0, 0)
    ),
    nn.Tanh()
)

last_block = nn.Sequential(
    nn.ConvTranspose2d(
        32, 1,
        kernel_size=(1, 1),
        stride=(1, 1),
        padding=(0, 0)
    ),
    nn.Tanh(),
    nn.Upsample(
        scale_factor=2,
        mode="nearest"
    )
)

In [ ]:
new_block[0].weight.size()

In [ ]:
new_block[0].weight[:, :, 1, 1]

In [ ]:
with th.no_grad():
    new_block[0].weight.zero_()
    new_block[0].weight[:, :, 1, 1] = th.eye(32, 32)

In [ ]:
new_block[0].weight[:, :, 1, 1]

In [ ]:
last_block[0].weight[:, :, 0, 0]

# Code Thierry

In [ ]:
"""
Created on Sun May 15 14:11:27 2022

@author: thierry
"""

import numpy as np

def multiple(A,p):
    
    n,q = A.shape
    if min(n,q)>p:
        print('On fait une ACP...')
        
    U,D,V = np.linalg.svd(A, full_matrices=True)

    DD = np.zeros(p)
    DD[:min([n,p,q])] = np.sqrt(D[:min([n,p,q])])
    D1 = np.zeros((n,p))
    np.fill_diagonal(D1,DD)
    D2 = np.zeros((p,q))  
    np.fill_diagonal(D2,DD)
    
    G = np.random.normal(size=(p,p))
    G = G+np.transpose(G)   
    E,W = np.linalg.eigh(G)
    
    B = np.matmul(U,np.matmul(D1,W))
    C = np.matmul(np.transpose(W),np.matmul(D2,V))    
    
    return(B,C)

In [ ]:
def fill_diag(m: th.Tensor, v: th.Tensor) -> th.Tensor:
    print(m.size())
    print(v.size())
    
    mask = th.diag(th.ones_like(v))
    full_mask = th.zeros(*m.size())
    start, end = min(full_mask.size()[0], mask.size()[0]), min(full_mask.size()[1], mask.size()[1])
    full_mask[:start, :end] = mask[:start, :end]
    
    diag = th.diag(v)
    full_diag = th.zeros(*m.size())
    start, end = min(full_diag.size()[0], diag.size()[0]), min(full_diag.size()[1], diag.size()[1])
    full_diag[:start, :end] = diag[:start, :end]
    
    return full_mask * full_diag + (1. - full_mask) * m

def decomposition(m: th.Tensor, p: int) -> Tuple[th.Tensor, th.Tensor]:
    n, q = m.size()[-2:]
    
    u, d, v = th.linalg.svd(m, full_matrices=True)
    
    dd = th.zeros(p)
    dd[:min(n, p, q)] = th.sqrt(d[:min(n, p, q)])
    
    d1 = th.zeros(n, p)
    d1 = fill_diag(d1, dd)
    
    d2 = th.zeros(p, q)
    d2 = fill_diag(d2, dd)
    
    g = th.randn(p, p)
    g = g + g.transpose(1, 0)
    e, w = th.linalg.eigh(g)
    
    b = u @ (d1 @ w)
    c = w.transpose(1, 0) @ (d2 @ v)
    
    return b, c

In [ ]:
m = th.randn(32, 2)

In [ ]:
m_1, m_2 = decomposition(m, 24)

In [ ]:
m

In [ ]:
def round(t: th.Tensor, decimals=0) -> th.Tensor:
    return (t * 10 ** decimals).round() / (10 ** decimals)

In [ ]:
round(m_1 @ m_2, decimals=4) == round(m, decimals=4)

In [ ]:
help(th.round)

In [ ]:
th.__version__

# Code Thierry

In [3]:
"""
Created on Sun May 15 14:11:27 2022

@author: thierry
"""

import numpy as np

def multiple(A,p):
    
    n,q = A.shape
    if min(n,q)>p:
        print('On fait une ACP...')
        
    U,D,V = np.linalg.svd(A, full_matrices=True)

    DD = np.zeros(p)
    DD[:min([n,p,q])] = np.sqrt(D[:min([n,p,q])])
    D1 = np.zeros((n,p))
    np.fill_diagonal(D1,DD)
    D2 = np.zeros((p,q))  
    np.fill_diagonal(D2,DD)
    
    G = np.random.normal(size=(p,p))
    G = G+np.transpose(G)   
    E,W = np.linalg.eigh(G)
    
    B = np.matmul(U,np.matmul(D1,W))
    C = np.matmul(np.transpose(W),np.matmul(D2,V))    
    
    return(B,C)

In [87]:
def fill_diag(m: th.Tensor, v: th.Tensor) -> th.Tensor:
    mask = th.diag(th.ones_like(v))
    full_mask = th.zeros(*m.size())
    start, end = min(full_mask.size()[0], mask.size()[0]), min(full_mask.size()[1], mask.size()[1])
    full_mask[:start, :end] = mask[:start, :end]
    
    diag = th.diag(v)
    full_diag = th.zeros(*m.size())
    start, end = min(full_diag.size()[0], diag.size()[0]), min(full_diag.size()[1], diag.size()[1])
    full_diag[:start, :end] = diag[:start, :end]
    
    return full_mask * full_diag + (1. - full_mask) * m

def decomposition(m: th.Tensor, p: int) -> Tuple[th.Tensor, th.Tensor]:
    n, q = m.size()[-2:]
    
    u, d, v = th.linalg.svd(m, full_matrices=True)
    
    d_diag = th.zeros(p)
    d_diag[:min(n, p, q)] = th.sqrt(d[:min(n, p, q)])
    
    d1 = th.zeros(n, p)
    d1 = fill_diag(d1, d_diag)
    
    d2 = th.zeros(p, q)
    d2 = fill_diag(d2, d_diag)
    
    g = th.randn(p, p)
    g = g + g.transpose(1, 0)
    e, w = th.linalg.eigh(g)
    
    b = u @ (d1 @ w)
    c = w.transpose(1, 0) @ (d2 @ v)
    
    return b, c

In [88]:
m = th.randn(64, 2)

In [89]:
m_1, m_2 = decomposition(m, 56)

In [5]:
def round(t: th.Tensor, decimals=0) -> th.Tensor:
    return (t * 10 ** decimals).round() / (10 ** decimals)

In [93]:
round(m_1 @ m_2, decimals=5) == round(m, decimals=5)

tensor([[False,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [False,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True, False],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True, 

In [92]:
help(th.round)

Help on built-in function round:

round(...)
    round(input, *, out=None) -> Tensor
    
    Returns a new tensor with each of the elements of :attr:`input` rounded
    to the closest integer.
    
    Args:
        input (Tensor): the input tensor.
    
    Keyword args:
        out (Tensor, optional): the output tensor.
    
    Example::
    
        >>> a = torch.randn(4)
        >>> a
        tensor([ 0.9920,  0.6077,  0.9734, -1.0362])
        >>> torch.round(a)
        tensor([ 1.,  1.,  1., -1.])



In [72]:
th.__version__

'1.10.2+cu113'

In [152]:
conv = nn.ConvTranspose2d(
    32, 32,
    kernel_size=(3, 3),
    stride=(2, 2),
    padding=(1, 1),
    output_padding=(1, 1)
)

In [158]:
nn.init.zeros_(conv.weight)
nn.init.zeros_(conv.bias)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [159]:
conv.weight.data[:, :, 1:, 1:] = (
    th.eye(32)[:, :, None, None]
    .repeat(1, 1, 2, 2)
)

In [160]:
conv.weight.data[:, :, 1, 1]

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [161]:
v = th.ones(1, 32, 4, 4)
o = conv(v)

In [162]:
o[0, 0]

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]], grad_fn=<SelectBackward0>)

In [ ]:
conv = nn.Conv2d(
    32, 32,
    kernel_size=(3, 3),
    stride=(2, 2),
    padding=(1, 1)
)

In [ ]:
nn.init.zeros_(conv.weight)
nn.init.zeros_(conv.bias)

In [ ]:
conv.weight.data[:, :, 1:, 1:] = (
    th.eye(32)[:, :, None, None]
    .repeat(1, 1, 2, 2) / 4
)

In [ ]:
x = th.ones(1, 32, 8, 8)

In [ ]:
o = conv(x)

In [ ]:
o

In [180]:
nn.init.zeros_(conv.weight)
nn.init.zeros_(conv.bias)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [181]:
conv.weight.data[:, :, 1:, 1:] = (
    th.eye(32)[:, :, None, None]
    .repeat(1, 1, 2, 2) / 4
)

In [182]:
x = th.ones(1, 32, 8, 8)

In [183]:
o = conv(x)

In [184]:
o

tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],

         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
    

# Code Sam

In [3]:
from music_gan.networks import matrix_multiple

import torch as th

In [4]:
a = th.randn(32, 2)
b, c = matrix_multiple(a, 24)

In [6]:
round(b @ c, decimals=5) == round(a, decimals=5)

tensor([[ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [False,  True],
        [ True, False],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [False,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True],
        [ True,  True]])

In [ ]:
a[round(b @ c, decimals=5) != round(a, decimals=5)]

In [ ]:
(b @ c)[round(b @ c, decimals=5) != round(a, decimals=5)]

In [8]:
a[round(b @ c, decimals=5) != round(a, decimals=5)]

tensor([ 2.1386, -1.1690, -0.5908])

In [9]:
(b @ c)[round(b @ c, decimals=5) != round(a, decimals=5)]

tensor([ 2.1386, -1.1690, -0.5908])